# CoE 163 2nd Semester AY 2022-2023
## Software Exercise 04
__IMPORTANT__: This notebook does not work as is because it is your obligation to fill in the missing parts. :) Required tasks include:

- Formulating appropriate kernels
- Invoking the appropriate kernels
- Writing a short journal at the end of this notebook

-----

This notebook instance contains a template notebook for the software exercise. This exercise aims to get you started with CUDA programming on Python.

Although Python is inherently not a language for parallel programming, it can still use interfaces to communicate to libraries that can. For this exercise, __Numba__ will be used as the wrapper for NVIDIA CUDA.

##Acknowledgement
This SE has been adapted from one of the exercises offered by CS 239 for 2nd Semester AY 2020-2021.

## References
Here are some references that can potentially help you finish this SE.

- [NVIDIA CUDA C Guide](https://docs.nvidia.com/cuda/cuda-c-programming-guide/)
- [CUDA Kernels in Numba](https://numba.pydata.org/numba-doc/latest/cuda/kernels.html)
- [CUDA Programming in Numba](https://nyu-cds.github.io/python-numba/05-cuda)
- [CUDA Thread Indexing](https://blog.usejournal.com/cuda-thread-indexing-fb9910cba084)


## Setup
Before playing with this notebook, you need to first enable the GPU by first looking at the toolbar and going to `Runtime > Change runtime type`. Find the Hardware accelerator subfield and choose "GPU". Click `Save` to save the changes. If you have ran some code before this change, rerun them to be able to recognize the existence of the GPU.

Numba is one of the several Python wrappers that can interface with CUDA. Import the appropriate API by executing `from numba import cuda`. To check whether the import was successful, the `__version__` property can be called.

In [ ]:
import sys
import time

import numpy as np
import numba

from numba import cuda

print("Python version:", sys.version)
print("Numba version:", numba.__version__)
print("Numpy version:", np.__version__)

Python version: 3.10.12 (main, Jun  7 2023, 12:45:35) [GCC 9.4.0]
Numba version: 0.56.4
Numpy version: 1.22.4


## Fetching Available GPUs
Call `nvidia-smi` on the command line to fetch the available GPUs. This command will then list hardware statistics of each available GPU in this instance.

In [ ]:
!nvidia-smi

Tue Jun 13 11:48:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Now, it's your time to list the available GPUs for our program. Write an appropriate method for calling the list of available GPUs using Numba. If the method used is correct, the subsequent looping code should be able to call the following properties as the elements of the resulting list is an object.

In [ ]:
# TODO: Replace this empty array with the appropriate Numba CUDA call
device_list = cuda.gpus

print("CUDA-supported GPUs available:")
for each_dev in device_list:
    print(f"{each_dev.id}: {each_dev.name} cc v{each_dev.compute_capability[0]}.{each_dev.compute_capability[1]}")

CUDA-supported GPUs available:
0: b'Tesla T4' cc v7.5


## A GPU Parallel Program (Part I)
A GPU parallel program consists of a main program and several "slave" programs named _kernels_. Each _kernel_ is a sort of routine that is started by the main program but runs independently. These kernels are usually simple routines that process an element of the input(s).

CUDA kernels are defined functions that are marked in some way. CUDA kernels  normally look like this.

```python
@cuda.jit
def kernel_self(a):
    a += 1

@cuda.jit
def kernel_out(out, a, b):
    out = a + b
```

Numba recognizes the `@cuda.jit` decorator to mean that the function below it is a CUDA kernel. The first kernel `kernel_self` shows a kernel that updates one of the input parameters. The second kernel `kernel_out` shows a kernel that dumps the result of one or more of the parameters into another resulting parameter. Note that these kernels _do not_ return an output because the output _should be_ redirected to one of the kernel parameters. This is possible because the underlying API that Numba uses is still the C/C++ one, which honors pass-by-reference invocations.

## Grids, Threads, and Oh My!
Developing a CUDA program involves knowledge on the concept of grids, threads, and blocks. When a kernel is executed, the execution actually involves creation of a certain amount of _threads_ that can be grouped into _blocks_. This collection of blocks is then called a _grid_. Each thread executes a copy of the kernel.

A rough definition of the terms is as follows:

__grid__ -> a collection of blocks

__block__ -> a collection of threads

__thread__ -> a single instance that executes _a copy_ of the kernel

You can think of this as finding an address of a house. The house is a _thread_, which can be addressed as a lot and a block. The lot number is the specific address of the house within a block, and the block, which is analogous to a thread _block_, is a collection of houses congregating on one large piece of land. These blocks and their accompanying roads can then be collectively fenced off as a gated community, which is analogous to a _grid_.

A kernel is actually self-aware of its location within the grid in terms of its grid, block, and thread index. The following properties can be called within a kernel to find its address.

```python
tx = cuda.threadIdx.x # 0-index thread index
bx = cuda.blockIdx.x # 0-index block index
bdim = cuda.blockDim.x # Shape of block

# Absolute address of thread within a single grid
pos = bx * bdim + tx
```

Note that the properties `threadIdx` and `blockIdx` need to have the property `x` appended to it to return an integer index. This is because CUDA supports _multi-dimensional_ blocks and grids. In a NumPy setting, this can be useful especially if 2D or 3D matrices are involved in calculations. For example, a kernel can accept a 3D matrix, which can be defined with a 3D thread dimension. The thread indices can then be called as `threadIdx.x`, `threadIdx.y`, and `threadIdx.z`.

This example below is similar to the addressing shown above except that this is for 2D blocks (and therefore, 2D threads).

```python
tx, ty = cuda.threadIdx.x, cuda.threadIdx.y # 0-index thread index
bx = cuda.blockIdx.x # 0-index block index
bdimx, bdimy = cuda.blockDim.x, cuda.blockDim.y # Shape of block

# Absolute address of thread within a single grid
pos = (bx * bdimx * bdimy) + (ty * bdimx) + tx
```

Noting the routines above, we can see that we have converted these grid, block, and thread indices into a single index denoting the absolute position of a thread within a grid. Think of it as all of the houses laid out in a single line and then assigned a number. Numba has a convenience method for finding such index.

```python
# The number in the method signifies the dimension of the
# block and grid as declared when the kernel is called.

# A tuple is returned if the dimension is greater than 1

cuda.grid(1) # 0-index absolute position within grid
```

There are times that CUDA may return an absolute index that is out of bounds with respect to the input or output parameters. Hence, bounds checking is needed to be able to properly execute such kernels.

For this exercise, we will not work with multiple grids - only blocks and threads.

## A GPU Parallel Program (Part II)
Now that we have knowledge of a collection of threads, we can create CUDA kernels that work with a specific subset of the input parameters. For example, the CUDA kernel below increments a 2D matrix by element.

```python
@cuda.jit
def increment(a):
    x = cuda.threadIdx.x
    y = cuda.threadIdx.y

    # The indices can also be called using this method
    # x, y = cuda.grid(2)

    if x < a.shape[0] and y < a.shape[1]:
        a[x, y] += 1
```

Notice that the routine above includes a line to check whether the indices returned by CUDA are within the bounds of the matrix.

## Matrix Addition Kernels
Now, it's your time to implement the different kernels for your simple program. Populate the three kernels with simple routines that add two matrices by element, by row, and by column. You should have a value for the number of blocks and threads in mind. Be careful of out-of-bound exceptions and make sure to use the appropriate grid, block, and thread size!

_HINT:_ It's OK to create these kernels for a single block. Note that we will only be operating on a single grid.

In [ ]:
# WARN: There may be missing decorators in this code, so check before running it.

"""Performs matrix addition element by element

This CUDA kernel adds each matching element in a and b
and dumps the result to each matching element in c.

The sizes of all three matrices should be the same.

Parameters
----------
res : numpy matrix
    Matrix resulting in the addition of a and b
a   : numpy matrix
    First matrix operand
b   : numpy matrix
    Second matrix operand
"""

@cuda.jit
def kernel_1t1e(res, a, b):
    # TODO: Populate this function
    # HINT: Add elements of a and b element by element
    #       and save the result to res.
    # HINT: The answer is less than 10 lines.

    #Let thread be row, block be col
    x = cuda.threadIdx.x    #Row
    y = cuda.threadIdx.y    #Col

    if x < res.shape[0] and y < res.shape[1]:   #Check array boundaries, row-column or depth-row-column?
      res[x][y] = a[x][y] + b[x][y]             #Element-wise addition

"""Performs matrix addition by matrix rows

This CUDA kernel loops through each row in a and b
and adds each matching e on each row. The resulting
row is dumped to the matching row in c.

The sizes of all three matrices should be the same.

Parameters
----------
res : numpy matrix
    Matrix resulting in the addition of a and b
a   : numpy matrix
    First matrix operand
b   : numpy matrix
    Second matrix operand
"""
@cuda.jit
def kernel_1t1r(res, a, b):
    # TODO: Populate this function
    # HINT: Iterate through each row of a and b together,
    #       then add the elements in the rows and save the
    #       result to res.
    # HINT: The answer is less than 10 lines.

    x = cuda.threadIdx.x    #Row
    y = cuda.threadIdx.y    #Col

    if x < res.shape[0]:                #Check boundaries
      for i in range(res.shape[1]):
        res[x][i] = a[x][i] + b[x][i]

"""Performs matrix addition by matrix columns

This CUDA kernel loops through each column in a and b
and adds each matching on each row. The resulting
column is dumped to the matching row in c.

The sizes of all three matrices should be the same.

Parameters
----------
res : numpy matrix
    Matrix resulting in the addition of a and b
a   : numpy matrix
    First matrix operand
b   : numpy matrix
    Second matrix operand
"""
@cuda.jit
def kernel_1t1c(res, a, b):
    # TODO: Populate this function
    # HINT: Iterate through each column of a and b together,
    #       then add the elements in the columns and save the
    #       result to res.
    # HINT: The answer is less than 10 lines.

    # Number of blocks = number of columnd
    # Threads/block = number of rows

    x = cuda.threadIdx.x    #Row
    y = cuda.threadIdx.y    #Col

    if y < res.shape[1]:                #Check boundaries
      for i in range(res.shape[0]):
        res[i][y] = a[i][y] + b[i][y]

## Running The Kernels
Running a kernel is simple - just execute the function like you would a normal Python function. However, it differs from a normal function because can also take additional parameters influencing the amount of kernels to execute.

A kernel can be called like this. Note that a list of parameters enclosed by brackets precede the actual parameters to the kernel.

```python
kernel[bpg, tpb](out, a, b)
```

The parameters `bpg` and `tpb` denote the number of _blocks_ (per grid) and _threads_ (per block), respectively. As stated before, each of these numbers can take up either a number or a tuple of up to three numbers. Some examples of this type of invocation for a group of 16 threads are as follows.

```python
# Single block
kernel[1, 16](out, a, b)
kernel[1, (4, 4)](out, a, b)
kernel[1, (8, 2)](out, a, b)

# Multiple blocks
kernel[4, 4](out, a, b)
kernel[2, 8](out, a, b)
```

Note that by default, Numba runs kernels synchronously. This means that the program executes the next line only when all of the threads has been executed successfully.

Now, it's your time to execute the kernels. Replace the appropriate comment stubs with your kernel calls. Be mindful of the number of threads and blocks to use for each call!

Once you have written the appropriate calls, you can run the stub code below. Make sure that all of the `Kernel x correct?` print outs are `True` to ensure that your kernels yield the correct answer to the addition.

_HINT:_ It's OK to execute the kernels for a single block. Note that we will only be operating on a single grid.

In [ ]:
# Edit this matrix size to experiment with its effect on kernel runtime
mat_size = (30, 30)

# Generate two random integer matrices and one empty matrix to hold the result
a, b = np.random.randint(-100, 100, mat_size), np.random.randint(-100, 100, mat_size)
c = np.zeros(mat_size)

# Generate reference answer to matrix addition
ctrl_ans = a + b

s_time = time.time()
# TODO: Replace this comment with kernel_1t1e call
kernel_1t1e[1, mat_size](c, a, b)
e_time = time.time()

print("Kernel 1t1e correct?", np.all(ctrl_ans == c))
print(f"Execution time: {e_time - s_time:.4f}s")

s_time = time.time()
# TODO: Replace this comment with kernel_1t1r call
kernel_1t1r[1, (1, mat_size[1])](c, a, b)
e_time = time.time()
print("Kernel 1t1r correct?", np.all(ctrl_ans == c))
print(f"Execution time: {e_time - s_time:.4f}s")

s_time = time.time()
# TODO: Replace this comment with kernel_1t1c call
kernel_1t1c[1, (mat_size[0], 1)](c, a, b)
e_time = time.time()
print("Kernel 1t1c correct?", np.all(ctrl_ans == c))
print(f"Execution time: {e_time - s_time:.4f}s")

Kernel 1t1e correct? True
Execution time: 0.0035s
Kernel 1t1r correct? True
Execution time: 0.0021s
Kernel 1t1c correct? True
Execution time: 0.0045s


## Which is Faster?
The subroutine above is actually the _main_ routine that manages all of the kernels. This also contains a simple runtime measurement to check which of these programs run the fastest. With the results above, write a short journal below this part that answers the following questions or considerations.

- Choice of number of threads and blocks, and reasoning behind such
- Rank of the three kernels in terms of speed
- Why the kernels ran with their appropriate speeds

__Write your journal here~!__

1. Choice of number of threads and blocks, and reasoning behind such

  -I opted to use a single block for all kernels as it is much easier to understand and program. For element-wise matrix addition, I used 2D threads in the form of (row, column) to represent the elements; and since the size of the square matrix used is 20, the tpb argument is set to (20, 20). For the per-row and per-column addition, I respectively used (1, 20) and (20, 1) threads.

2. Rank of the three kernels in terms of speed

  -Fastest- 1t1r, mid- 1t1c, slowest- 1t1e; but 1t1c and 1t1r runtimes were close based on the three executions done

  -1t1r runtimes- 0.0018s, 0.0026s, 0.0019s

  -1t1c runtimes- 0.0019s, 0.0027s, 0.0022s

  -1t1e runtimes- 0.0025s, 0.0026s, 0.0023s

3. Why the kernels ran with their appropriate speeds

  -I think that the reason why the element-wise addition generally ran slower is because it involves more operations. 1t1r ran faster than 1t1c as Google Colab is row-major.